## Your First NLP Application

* **Building a sentiment analyzer using AllenNLP**
* **Applying basic machine learning conceopts (datasets, classification, and regression)**
* **Employing neural network concepts (word embeddings, recurrent neural networks, linear layers)**
* **Training the model through reducing loss**
* **Evaluating and deploying your model**

### 2.1 Introducing Sentiment Analysis
![alt text for screen readers](figure2.1.PNG "Text to show on mouseover")

### 2.2 Working with NLP datasets

##### 2.2.1 What is a dataset?
* A dataset simply means a collection of data.
* In NLP, records in a dataset are usually some type of linguistic units, such as words,
sentences, or documents. A dataset of natural language texts is called a corpus (plural:
corpora).
* A closely related term to a record is an instance. In machine learning, an instance is
a basic unit for which the prediction is made.
* A label is a piece of information attached to some linguistic unit in a dataset.

<img src="figure2.2.PNG" alt="MarineGEO circle logo" style="height: 400px; width:900px;"/>



##### 2.2.2 Stanford Sentiment Treebank

Data Link: https://nlp.stanford.edu/sentiment/

##### 2.2.3 Train, Validation, and Test sets

* A train (or training) set is the main dataset used to train the NLP/ML models.
* A validation set (also called a dev or development set) is used for model selection. The validation set is also used for tuning hyperparameters. 
* Finally, a test set is used to evaluate the model using a new, unseen set of data.

<img src="figure2.3.PNG" alt="MarineGEO circle logo" style="height: 400px; width:700px;"/>

##### 2.2.4 Loading SST datasets using AllenNLP

In [1]:
! git clone https://github.com/mhagiwara/realworldnlp.git

Cloning into 'realworldnlp'...
fatal: '$GIT_DIR' too big


In [6]:
from itertools import chain
from typing import Dict

import numpy as np
import torch
import torch.optim as optim
from allennlp.data.data_loaders import MultiProcessDataLoader
from allennlp.data.samplers import BucketBatchSampler
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.nn.util import get_text_field_mask
from allennlp.training import GradientDescentTrainer
from allennlp.training.metrics import CategoricalAccuracy, F1Measure
from allennlp_models.classification.dataset_readers.stanford_sentiment_tree_bank import \
    StanfordSentimentTreeBankDatasetReader

from predictors import SentenceClassifierPredictor

In [7]:
# import sys
# path = "C:\\Users\\longln3\\OneDrive - Hanoi University of Science and Technology\\Real-World-Natural-Language-Processing-Practical-applications-with-deep-learning-Masato-Hagiwara-\\realworldnlp"
# sys.path.append(path)
# from realworldnlp.realworldnlp import SentenceClassifier

In [3]:
EMBEDDING_DIM = 128
HIDDEN_DIM = 128

### 2.3 Using word embeddings

##### 2.3.1 What are word embeddings?

* Word embeddings are one of the most important concepts in modern NLP.
* Technically, an embedding is a continuous vector representation of something that is usually discrete.
* A word embedding is acontinuous vector representation of a word.

##### 2.3.2 Using word embeddings for sentiment analysis

In [4]:
reader = StanfordSentimentTreeBankDatasetReader()
train_path = './trees/train.txt'
dev_path = './trees/dev.txt'

In [5]:
sampler = BucketBatchSampler(batch_size=32, sorting_keys=["tokens"])
train_data_loader = MultiProcessDataLoader(reader, train_path, batch_sampler=sampler)
dev_data_loader = MultiProcessDataLoader(reader, dev_path, batch_sampler=sampler)

loading instances: 0it [00:00, ?it/s]

loading instances: 0it [00:00, ?it/s]

In [6]:
# You can optionally specify the minimum count of tokens/labels.
# `min_count={'tokens':3}` here means that any tokens that appear less than three times
# will be ignored and not included in the vocabulary.
vocab = Vocabulary.from_instances(chain(train_data_loader.iter_instances(), dev_data_loader.iter_instances()),
                                  min_count={'tokens': 3})

building vocab: 0it [00:00, ?it/s]

In [7]:
train_data_loader.index_with(vocab)
dev_data_loader.index_with(vocab)

In [8]:
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)

In [9]:
# BasicTextFieldEmbedder takes a dict - we need an embedding just for tokens,
# not for labels, which are used as-is as the "answer" of the sentence classification
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

### 2.4 Neural Networks

In [10]:
# Model in AllenNLP represents a model that is trained.
class LstmClassifier(Model):
    def __init__(self,
                 embedder: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 vocab: Vocabulary,
                 positive_label: str = '4') -> None:
        super().__init__(vocab)
        # We need the embeddings to convert word IDs to their vector representations
        self.embedder = embedder

        self.encoder = encoder

        # After converting a sequence of vectors to a single vector, we feed it into
        # a fully-connected linear layer to reduce the dimension to the total number of labels.
        self.linear = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))

        # Monitor the metrics - we use accuracy, as well as prec, rec, f1 for 4 (very positive)
        positive_index = vocab.get_token_index(positive_label, namespace='labels')
        self.accuracy = CategoricalAccuracy()
        self.f1_measure = F1Measure(positive_index)

        # We use the cross entropy loss because this is a classification task.
        # Note that PyTorch's CrossEntropyLoss combines softmax and log likelihood loss,
        # which makes it unnecessary to add a separate softmax layer.
        self.loss_function = torch.nn.CrossEntropyLoss()

    # Instances are fed to forward after batching.
    # Fields are passed through arguments with the same name.
    def forward(self,
                tokens: Dict[str, torch.Tensor],
                label: torch.Tensor = None) -> torch.Tensor:
        # In deep NLP, when sequences of tensors in different lengths are batched together,
        # shorter sequences get padded with zeros to make them equal length.
        # Masking is the process to ignore extra zeros added by padding
        mask = get_text_field_mask(tokens)

        # Forward pass
        embeddings = self.embedder(tokens)
        encoder_out = self.encoder(embeddings, mask)
        logits = self.linear(encoder_out)

        # In AllenNLP, the output of forward() is a dictionary.
        # Your output dictionary must contain a "loss" key for your model to be trained.
        output = {"logits": logits}
        if label is not None:
            self.accuracy(logits, label)
            self.f1_measure(logits, label)
            output["loss"] = self.loss_function(logits, label)

        return output

    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {'accuracy': self.accuracy.get_metric(reset),
                **self.f1_measure.get_metric(reset)}

In [11]:
# Seq2VecEncoder is a neural network abstraction that takes a sequence of something
# (usually a sequence of embedded word vectors), processes it, and returns a single
# vector. Oftentimes this is an RNN-based architecture (e.g., LSTM or GRU), but
# AllenNLP also supports CNNs and other simple architectures (for example,
# just averaging over the input vectors).
encoder = PytorchSeq2VecWrapper(
    torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

In [12]:
model = LstmClassifier(word_embeddings, encoder, vocab)

In [13]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [14]:
trainer = GradientDescentTrainer(
    model=model,
    optimizer=optimizer,
    data_loader=train_data_loader,
    validation_data_loader=dev_data_loader,
    patience=10,
    num_epochs=20,
    cuda_device=-1)

trainer.train()

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/267 [00:00<?, ?it/s]

...

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

{'best_epoch': 5,
 'peak_worker_0_memory_MB': 0,
 'training_duration': '0:06:30.002704',
 'epoch': 15,
 'training_accuracy': 0.7813670411985019,
 'training_precision': 0.8778452277183533,
 'training_recall': 0.8982919454574585,
 'training_f1': 0.8879508972167969,
 'training_loss': 0.5738753581538183,
 'training_worker_0_memory_MB': 0.0,
 'validation_accuracy': 0.3614895549500454,
 'validation_precision': 0.3563218414783478,
 'validation_recall': 0.3757575750350952,
 'validation_f1': 0.3657817244529724,
 'validation_loss': 2.3366266523088726,
 'best_validation_accuracy': 0.335149863760218,
 'best_validation_precision': 0.30249109864234924,
 'best_validation_recall': 0.5151515007019043,
 'best_validation_f1': 0.38116592168807983,
 'best_validation_loss': 1.5144107137407576}

In [15]:
predictor = SentenceClassifierPredictor(model, dataset_reader=reader)
logits = predictor.predict('This is the best movie ever!')['logits']
label_id = np.argmax(logits)

print(model.vocab.get_token_from_index(label_id, 'labels'))

Spacy models 'en_core_web_sm' not found.  Downloading and installing.


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
4
